<a href="https://colab.research.google.com/github/OswaldGutierrez/Modelos-IA-2025-1/blob/main/04%20-%20modelo%20con%20preprocesado%20y%20regresi%C3%B3n%20log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="6">Modelo con preprocesado y Regresión Logística</font>

**OSWALD DAVID GUTIÉRREZ CORTINA**

In [2]:
!pip install kaggle

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import time

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


<font size="5">**1. Descargamos los archivos desde mi repositorio GitHub**</font>


In [3]:
!wget https://raw.githubusercontent.com/OswaldGutierrez/contenidoTrain/main/train.zip
!unzip train.zip
df_train  = pd.read_csv("train.csv")


!wget https://raw.githubusercontent.com/OswaldGutierrez/contenidoTest/main/test.zip
!unzip test.zip
df_test  = pd.read_csv("test.csv")

--2025-07-04 02:36:35--  https://raw.githubusercontent.com/OswaldGutierrez/contenidoTrain/main/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19235312 (18M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  18.34M  --.-KB/s    in 0.1s    

2025-07-04 02:36:36 (150 MB/s) - ‘train.zip’ saved [19235312/19235312]

Archive:  train.zip
  inflating: train.csv               
--2025-07-04 02:36:40--  https://raw.githubusercontent.com/OswaldGutierrez/contenidoTest/main/test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, 

<font size="5">**2. Dimensiones de los Dataframes**</font>


In [4]:
print ("shape of loaded dataframe", df_train.shape)
print ("shape of loaded dataframe", df_test.shape)

shape of loaded dataframe (692500, 21)
shape of loaded dataframe (296786, 20)


<font size="5">**3. Función clean_data**</font>

Se encarga de reemplazar los valores faltantes por la moda. Además, se normalizan los valores y se toma una columna como objetivo: ESTU_PRGM_ACADEMICO


In [16]:
def limpiarData(zCopia):
    # Hacemos una copia para trabajar sobre ella
    zCopia = zCopia.copy()

    # Obtenemos las columnas categóricas.
    ccols = [col for col in zCopia.columns if zCopia[col].dtype == 'object' and col != 'ID']
    print("Categorical columns before detailed processing:", ccols)

    # Cambiamos los datos faltantes con los que más se repiten (moda)
    for column in ccols:
        if column in zCopia.columns:
            valorComun = zCopia[column].mode()[0]
            zCopia[column].fillna(valorComun, inplace=True)

    # Normalizamos (Pasamos valores de string a enteros)
    zCopia['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({
        'No pagó matrícula': 0,
        'Menos de 500 mil': 1,
        'Entre 500 mil y menos de 1 millón': 2,
        'Entre 1 millón y menos de 2.5 millones': 3,
        'Entre 2.5 millones y menos de 4 millones': 4,
        'Entre 4 millones y menos de 5.5 millones': 5,
        'Entre 5.5 millones y menos de 7 millones': 6,
        'Más de 7 millones': 7
    }, inplace=True)

    zCopia['ESTU_HORASSEMANATRABAJA'].replace({
        '0': 0,
        'Menos de 10 horas': 1,
        'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3,
        'Más de 30 horas': 4
    }, inplace=True)

    zCopia['FAMI_ESTRATOVIVIENDA'].replace({
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6,
        'Sin Estrato': 0
    }, inplace=True)

    siNoCols = [
        "FAMI_TIENEINTERNET",
        "ESTU_PAGOMATRICULAPROPIO",
        "FAMI_TIENELAVADORA",
        "FAMI_TIENEAUTOMOVIL",
        "ESTU_PRIVADO_LIBERTAD",
        "FAMI_TIENECOMPUTADOR",
        "FAMI_TIENEINTERNET.1"
    ]
    for col in siNoCols:
        if col in zCopia.columns:
            zCopia[col] = zCopia[col].astype(str)
            zCopia[col].replace({'Si': 1, 'No': 0}, inplace=True)
            zCopia[col] = pd.to_numeric(zCopia[col], errors='coerce')
            zCopia[col].fillna(0, inplace=True)


    # Limpiamos la columna objetivo 'ESTU_PRGM_ACADEMICO'
    zCopia['ESTU_PRGM_ACADEMICO'] = zCopia['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN ', '', regex=False)
    zCopia['ESTU_PRGM_ACADEMICO'] = zCopia['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL DE ', '', regex=False)
    zCopia['ESTU_PRGM_ACADEMICO'] = zCopia['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL ', '', regex=False)
    zCopia['ESTU_PRGM_ACADEMICO'] = zCopia['ESTU_PRGM_ACADEMICO'].str.replace('PROGRAMA ', '', regex=False)

    replacements = {
        'INGE': 'INGENIERIA', 'LIC': 'LICENCIATURA', 'Lic': 'LICENCIATURA',
        'ADM': 'ADMINISTRACION', 'CONT': 'CONTADURIA', 'COMU': 'COMUNICACION',
        'DEPORT': 'DEPORTE', 'PSIC': 'PSICOLOGIA', 'TRABAJO SOCIAL': 'TRABAJO SOCIAL',
        'DISE': 'DISEÑO', 'TERAP': 'TERAPIA', 'GASTRON': 'GASTRONOMIA',
        'ZOOTECNIA': 'VETERINARIA Y ZOOTECNIA', 'NUTRIC': 'NUTRICION',
        'MARKETING': 'MARKETING', 'PUBLICIDAD': 'MARKETING', 'MERCAD': 'MARKETING',
        'MATE': 'MATEMATICAS Y ESTADISTICA', 'ESTA': 'MATEMATICAS Y ESTADISTICA',
        'ARTE': 'ARTES', 'PERIOD': 'PERIODISMO', 'LITER': 'LITERATURA',
        'MUS': 'MUSICA', 'MÚS': 'MUSICA', 'ECONO': 'ECONOMIA Y NEGOCIOS',
        'NEG': 'ECONOMIA Y NEGOCIOS', 'COME': 'ECONOMIA Y NEGOCIOS', 'FIN': 'ECONOMIA Y NEGOCIOS',
        'GEO': 'CIENCIAS DE LA TIERRA', 'CINE': 'CIENCIAS AUDIOVISUALES',
        'RADIO': 'CIENCIAS AUDIOVISUALES', 'TELEV': 'CIENCIAS AUDIOVISUALES',
        'AUDIOV': 'CIENCIAS AUDIOVISUALES', 'FOTOG': 'CIENCIAS AUDIOVISUALES',
        'DERECHO': 'DERECHO', 'POLIT': 'CIENCIAS POLITICAS', 'POLÍT': 'CIENCIAS POLITICAS',
        'QUIM': 'QUIMICA', 'QUÍM': 'QUIMICA', 'QU¿M': 'QUIMICA',
        'CONST': 'CONSTRUCCION', 'TURIS': 'TURISMO', 'BIBLIO': 'BIBLIOTECOLOGIA',
        'FILOS': 'FILOSOFIA', 'FILOL': 'FILOLOGIA', 'ODONTOLOG': 'ODONTOLOGIA',
        'BIOL': 'BIOLOGIA Y MICROBIOLOGIA', 'ECOL': 'ECOLOGIA', 'BACT': 'BACTERIOLOGIA',
        'CULT': 'CIENCIAS CULTURALES', 'INSTRUMENTACION': 'INSTRUMENTACION QUIRURGICA',
        'MILIT': 'CIENCIAS MARITIMAS Y AERONAUTICAS', 'NAVALES': 'CIENCIAS MARITIMAS Y AERONAUTICAS',
        'NAU': 'CIENCIAS MARITIMAS Y AERONAUTICAS'
    }

    # Hacemos un remplazo
    def aplicarRemplazosProgramas(text):
        for key, value in replacements.items():
            if key in text:
                return value
        return text

    zCopia['ESTU_PRGM_ACADEMICO'] = zCopia['ESTU_PRGM_ACADEMICO'].apply(aplicarRemplazosProgramas)

    # Agrupamos otras categorias
    conteoDeValores = zCopia['ESTU_PRGM_ACADEMICO'].value_counts()
    valoresMenores30 = conteoDeValores[conteoDeValores <= 30].index
    zCopia.loc[zCopia['ESTU_PRGM_ACADEMICO'].isin(valoresMenores30), 'ESTU_PRGM_ACADEMICO'] = 'OTRO PROGRAMA'

    # Aplicamos  one-hot encoding (Similar a la entrega #2)
    zCopia = pd.get_dummies(zCopia, columns=["ESTU_PRGM_ACADEMICO", "ESTU_PRGM_DEPARTAMENTO", "FAMI_EDUCACIONPADRE", "FAMI_EDUCACIONMADRE"], dtype=int)


    return zCopia

<font size="5">**4. Limpiamos nuestros datos**</font>


In [17]:
lentr = len(df_train)

trainLabels = df_train[['ID', 'RENDIMIENTO_GLOBAL']]

# Definir las columnas de origen, excluyendo la variable objetivo
colsOrigen = [i for i in df_train.columns if i != "RENDIMIENTO_GLOBAL"]

# Combinar los datos de entrenamiento y prueba para preprocesamiento
dataFull = pd.concat((df_train[colsOrigen], df_test[colsOrigen]))
dataFull.index = range(len(dataFull))

# Limpiar los datos usando la función limpiarData
dataFull = limpiarData(dataFull)

# Normaliamos valores
for col in dataFull.columns:
    if dataFull[col].dtype == 'bool':
        dataFull[col] = dataFull[col].astype(int)

# Dividir los datos combinados nuevamente en conjuntos de entrenamiento y prueba
trainOks = dataFull[dataFull['ID'].isin(trainLabels['ID'])]
test_listos = dataFull[~dataFull['ID'].isin(trainLabels['ID'])]

trainOks = trainOks.merge(trainLabels, on='ID', how='left')

Categorical columns before detailed processing: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']


/tmp/ipython-input-16-1410235025.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  zCopia[column].fillna(valorComun, inplace=True)
/tmp/ipython-input-16-1410235025.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zCopia['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({
/tmp/ipython-input-16-1410235

In [18]:
# Convertimos las 4 categorias de rendimiento a valores enteros
labelEncoder = LabelEncoder()
trainOks['RENDIMIENTO_GLOBAL'] = labelEncoder.fit_transform(trainOks['RENDIMIENTO_GLOBAL'])

In [19]:

# Dividimos la base de datos train en entrenamiento y testeo
X = trainOks.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
y = trainOks['RENDIMIENTO_GLOBAL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<font size="5">**5. Entrenamiento del modelo**</font>


In [20]:
# Realizamos el entrenamiento.
start_time = time.time()

modeloLogs = LogisticRegression(
    multi_class='multinomial',
    solver='saga',
    max_iter=200,
    n_jobs=-1,
    random_state=42
)

modeloLogs.fit(X_train, y_train)
end_time = time.time()

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [21]:
# Predecir en el conjunto de prueba
y_pred = modeloLogs.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Tiempo de entrenamiento: {end_time - start_time} segundos')

Accuracy: 0.3439061371841155
Tiempo de entrenamiento: 620.2924485206604 segundos
